In [ ]:
#libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from matplotlib.colors import ListedColormap
import seaborn as sns

In [ ]:
#read the file
raw_data=pd.read_csv("../input/gender-classification-dataset/gender_classification_v7.csv")

genders=raw_data.gender.values
genders=np.array(genders)
genders=np.where(genders=='Male',1,genders)
genders=np.where(genders=='Female',2,genders)
genders=genders.astype('int')
data=raw_data[["long_hair","forehead_width_cm","forehead_height_cm","nose_wide","nose_long","lips_thin","distance_nose_to_lip_long"]].values

data=MinMaxScaler(feature_range=(-1,1)).fit(data).transform(data)

#divide into train and test
data_train, data_test, genders_train, genders_test = train_test_split(data, genders, test_size = 0.25, random_state =0)


In [ ]:
#Valori pentru parametrii C si gamma
C_vect = [1,10,100]
gamma_vect = [0.0001,0.01,1]

Accmax = 0
Best_params = ""
for idx,kernel in enumerate(('linear', 'rbf', 'poly','sigmoid')):
    for i in range(3):
        for j in range(3):
            print("****************START******************")
            print(('Kernel = ' + str(kernel) + ' C = ' + (str(C_vect[i])) + ' Gamma = ' + (str(gamma_vect[j]))))
            
            svc = SVC(C=C_vect[i],gamma=gamma_vect[j],kernel=kernel,probability=True)
            
            svc.fit(data_train_scaled, genders_train)
            
            genders_pred_svc =svc.predict(data_test_scaled)
            c_m = confusion_matrix(genders_test, genders_pred_svc)
            
            print(c_m)
            ax=plt.matshow(c_m,cmap=plt.cm.Reds)
            print('Confusion matrix plot of SVC' + kernel + ' C = ' + str(C_vect[i]) + ' Gamma = ' + str(gamma_vect[j]))
            plt.colorbar(ax)
            plt.xlabel('Predicted')
            plt.ylabel('True')
            plt.show()
            
            f, ax = plt.subplots(figsize = (5,5))
            sns.heatmap(c_m, annot = True, linewidth = 0.5, linecolor = "red", fmt = ".0f", ax = ax)
            plt.show()
            
            # classification report
            print('\n Classification report for ' + kernel + ' C = ' + str(C_vect[i]) + ' Gamma = ' + str(gamma_vect[j]) + '\n',classification_report(genders_test, genders_pred_svc))
            
            TN = c_m[0][0]
            FN = c_m[1][0]
            TP = c_m[1][1]
            FP = c_m[0][1]
            
            # Miss Alarm Rate
            MAR = 100*FN/(TP+FN)
            
            # False alarm rate
            FAR = 100*FP/(TN+FN)
            
            # Overall accuracy
            ACC = 100*(TP+TN)/(TP+FP+FN+TN)
            if ACC > Accmax :
                Accmax = ACC
                Best_params = kernel + " C =" + str(C_vect[i]) + " gamma = " + str(gamma_vect[j])
            print('Precizia generala = ' + str(ACC) + '\nRata de alarme false = ' + str(FAR) + '\n Rata de alarme ratate = ' + str(MAR))
            print("****************FINISH******************")
            
            #Grafice
            #svc.fit(data_train[:,0:2], genders_train)
            #cmap_light = ListedColormap(['#AAFFAA','#FFAAAA'])
            #cmap_bold = ListedColormap(['#0000FF','#FF0000'])
            
            # creating a meshgrid
            #x_min, x_max = data[:, 0].min() - 1, data[:, 0].max() + 1
            #y_min, y_max = data[:, 1].min() - 1, data[:, 1].max() + 1
            #h=0.05
            #xx, yy = np.meshgrid(np.arange(x_min, x_max, h),np.arange(y_min, y_max, h))
            #xy_mesh=np.c_[xx.ravel(), yy.ravel()]
            #Z = svc.predict(xy_mesh)
            #Z = Z.reshape(xx.shape)
            
            #plotting data on decision boundary
            #plt.figure()
            #plt.pcolormesh(xx, yy, Z, cmap=cmap_light,shading='auto')
            #plt.scatter(data[:, 0], data[:, 1], c=genders, cmap=cmap_bold)
            #plt.xlim(xx.min(), xx.max())
            #plt.ylim(yy.min(), yy.max())
            #plt.xlabel('PC1');plt.ylabel('PC2')
            #plt.title('Kernel=' + kernel + ' C=' + str(C_vect[i]) + ' Gamma=' + str(gamma_vect[j]))
            #plt.show()